In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
def trim_string(row):

    row = row.split(maxsplit=first_n_words)
    row = ' '.join(row[:first_n_words])

    return row

In [4]:
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [5]:
% cd root

[Errno 2] No such file or directory: 'root'
/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [6]:
train_test_ratio = 0.90
train_valid_ratio = 0.80
first_n_words=250
# "./Models/Experiments/Claims/3epochs/real_fake_claims.csv"
data_path = "./Models/Experiments/Claims/3epochs"

In [18]:
# Read raw data
df_raw = pd.read_csv(data_path+"/real_fake_claims.csv")

# Trim text and titletext to first_n_words
df_raw['Claims'] = df_raw['Claims'].apply(trim_string)

# Split according to label
df_real = df_raw[df_raw['Label'] == 0]
df_fake = df_raw[df_raw['Label'] == 1]



In [19]:
# we need to preprocess such that the fake claims end at <EOS> to get a single full claim
def trim_claims(row):
    sentences = row.split('<EOS>')
    return sentences[0]

df_fake['Claims'] = df_fake['Claims'].apply(trim_claims)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
# example
np.asarray(df_fake["Claims"])[2]

'5. The display device of claim 1, wherein the second dielectric layer is arranged at a top surface, the first dielectric layer is arranged at a bottom surface, an insulating layer is provided on the top surface of the first dielectric layer, and an insulating material is provided on the insulating material layer; and wherein the first dielectric layer and the second dielectric layer comprise silicon (SiO). '

In [26]:
# Train-test split
df_real_full_train, df_real_test = train_test_split(df_real, train_size = train_test_ratio, random_state = 1)
df_fake_full_train, df_fake_test = train_test_split(df_fake, train_size = train_test_ratio, random_state = 1)

# Train-valid split
df_real_train, df_real_valid = train_test_split(df_real_full_train, train_size = train_valid_ratio, random_state = 1)
df_fake_train, df_fake_valid = train_test_split(df_fake_full_train, train_size = train_valid_ratio, random_state = 1)

# Concatenate splits of different labels
df_train = pd.concat([df_real_train, df_fake_train], ignore_index=True, sort=False)
df_valid = pd.concat([df_real_valid, df_fake_valid], ignore_index=True, sort=False)
df_test = pd.concat([df_real_test, df_fake_test], ignore_index=True, sort=False)

# Write preprocessed data
df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')].sample(frac= 1).to_csv(data_path + '/train.csv', index=False)
df_valid.loc[:, ~df_valid.columns.str.contains('^Unnamed')].sample(frac= 1).to_csv(data_path + '/valid.csv', index=False)
df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')].sample(frac= 1).to_csv(data_path + '/test.csv', index=False)

In [27]:
train = pd.read_csv(data_path + '/train.csv')
train

,Claims,Label
0,9. The method of claim 1 further comprising fo...,1
1,6. The light emitting diode package according ...,1
2,10. The method for manufacturing the OLED devi...,0
3,5. The semiconductor device according to claim...,0
4,"19. The method of claim 17, wherein a differen...",0
...,...,...
12119,7. The method for manufacturing a light emitti...,1
12120,"19. The surface treatment method of claim 14, ...",0
12121,"3. The device according to claim 1, wherein th...",1
12122,"14. The method of claim 11, wherein the formin...",0
